In [14]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from src.get_data import get_traffic_locations_df

In [3]:
pmed_trafico_ubicaciones = get_traffic_locations_df(data_dir="../01-data")
traffic_df_interim = pd.read_feather("../01-data/interim/traffic_df_interim.feather")

In [53]:
location_dataframe = pd.read_excel("../01-data/raw/ubicaciones.xlsx")
location_dataframe["nombre"] = np.where(location_dataframe.nombre.isnull(),location_dataframe.cod_cent,location_dataframe.nombre)
traffic_info_df = location_dataframe[["cod_cent","nombre","id"]]
traffic_info_df = traffic_info_df.drop_duplicates(subset="nombre")
traffic_info_df = traffic_info_df.append(pmed_trafico_ubicaciones[["cod_cent","nombre"]]).drop_duplicates(subset=["nombre"])
locations_dict = traffic_info_df.set_index("cod_cent").nombre.to_dict()
traffic_info_df

,cod_cent,nombre,id
0,01001,Jose Ortega y Gasset E-O - Pº Castellana-Serrano,3840.0
1,01002,Jose Ortega y Gasset O-E - Serrano-Pº Castellana,3841.0
2,01003,Pº Recoletos N-S - Almirante-Prim,3842.0
3,01004,Pº Recoletos S-N - Pl. Cibeles- Recoletos,3843.0
4,01005,(AFOROS) Pº Castellana S-N - Eduardo Dato - P...,3844.0
...,...,...,...
5099,17309,(MICRO) QUINTANA E-O - DELANTE GRUPO 10,NaN
5106,30300,(MICRO) PUENTE FRANCESES INTERNO(DETRÃS GRUP...,NaN
5107,31068,(TACTICO) ANTONIO MACHADO S-N - GIRO ISLA DE OZA,NaN
5108,32053,(TACTICO) SAN AQUILINO N-S - SAN BENITO-AV. AS...,NaN


Añadir el codigo a el dataset de trafico para identificar con las ubicaciones

In [60]:
traffic_df = traffic_df_interim.join(
    traffic_info_df.set_index("nombre"),
    on="nombre",how="left"
).reset_index(
    drop=True
).astype(
    {"fecha":"datetime64"}
).rename(columns={"fecha":"time"})
traffic_df["cod_cent"] = traffic_df["cod_cent"].fillna(traffic_df.nombre)
traffic_df = traffic_df[traffic_df.cod_cent.isin(pmed_trafico_ubicaciones.cod_cent)].drop_duplicates(["time","cod_cent"])
traffic_df.sort_values(by=["time","cod_cent"])

,nombre,time,intensidad,ocupacion,vmed,periodo_integracion,carga,cod_cent,id
114352334,03FL08PM01,2015-01-01 00:00:00,177.00,-0.25,60.25,9.50,0.00,03FL08PM01,1047.0
114360952,03FL08PM02,2015-01-01 00:00:00,65.25,-0.50,44.25,4.75,0.00,03FL08PM02,1046.0
114369570,03FL20PM01,2015-01-01 00:00:00,252.00,-0.25,64.75,4.75,0.00,03FL20PM01,6836.0
114378188,03FL56PM01,2015-01-01 00:00:00,333.75,0.00,52.75,4.75,0.00,03FL56PM01,1010.0
114386806,03FT08PM01,2015-01-01 00:00:00,245.25,0.00,63.25,4.75,0.00,03FT08PM01,1044.0
...,...,...,...,...,...,...,...,...,...
92176432,PM43012,2021-09-30 23:00:00,386.75,1.75,63.75,15.00,24.00,PM43012,6803.0
92214072,PM43032,2021-09-30 23:00:00,196.25,4.00,66.50,15.00,14.25,PM43032,6806.0
92240021,PM43151,2021-09-30 23:00:00,1256.00,4.75,62.50,15.00,26.50,PM43151,6807.0
92273945,PM43152,2021-09-30 23:00:00,68.25,1.00,33.50,15.00,5.50,PM43152,6808.0


Establecer latitud y longitud de cada punto

In [8]:
# Establecer latitud y longitud de cada punto
df = traffic_df.join(
    pmed_trafico_ubicaciones\
        .set_index("cod_cent")\
        .filter(items=['latitud','longitud']) ,
    on='cod_cent',
    how='left',
).dropna(subset=['latitud','longitud']).astype({"fecha":"datetime64"})
df

,nombre,fecha,intensidad,ocupacion,vmed,periodo_integracion,carga,cod_cent,id,latitud,longitud
0,C/. Fuente Carrantona - Paular-José Bergamín,2016-04-01 04:00:00,32.50,0.00,0.0,15.00,0.50,65016,6432,40.409479,-3.63018
1,C/. Fuente Carrantona - Paular-José Bergamín,2016-04-01 05:00:00,53.25,0.00,0.0,15.00,0.75,65016,6432,40.409479,-3.63018
2,C/. Fuente Carrantona - Paular-José Bergamín,2016-04-01 06:00:00,479.50,4.50,0.0,14.50,13.00,65016,6432,40.409479,-3.63018
3,C/. Fuente Carrantona - Paular-José Bergamín,2016-04-01 07:00:00,2057.25,29.00,0.0,15.00,64.00,65016,6432,40.409479,-3.63018
4,C/. Fuente Carrantona - Paular-José Bergamín,2016-04-01 09:00:00,1825.75,21.00,0.0,15.00,61.75,65016,6432,40.409479,-3.63018
...,...,...,...,...,...,...,...,...,...,...,...
113561899,Ángel Luis Herrán - Valdetorres del Jarama-Lóp...,2021-09-30 19:00:00,151.75,7.75,0.0,15.00,31.00,56028,6308,40.469907,-3.65075
113561900,Ángel Luis Herrán - Valdetorres del Jarama-Lóp...,2021-09-30 20:00:00,121.00,6.25,0.0,14.75,25.50,56028,6308,40.469907,-3.65075
113561901,Ángel Luis Herrán - Valdetorres del Jarama-Lóp...,2021-09-30 21:00:00,93.75,6.00,0.0,14.75,20.75,56028,6308,40.469907,-3.65075
113561902,Ángel Luis Herrán - Valdetorres del Jarama-Lóp...,2021-09-30 22:00:00,51.75,0.50,0.0,13.25,9.25,56028,6308,40.469907,-3.65075
